In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
country_df = spark.read.format("delta").load(f"{silver_folder_path}/country")

In [0]:
country_movie_df = spark.read.format("delta").load(f"{silver_folder_path}/production_country").filter(f"file_date = '{v_file_date}'")

In [0]:
join_country_df = country_df.join(country_movie_df,
                                  country_df.country_id == country_movie_df.country_id)\
                                      .select(country_df.country_name, country_movie_df.movie_id)

In [0]:
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies").filter(f"file_date = '{v_file_date}'")

movies_df = movies_df.filter("year_release_date >= 2010")
display(movies_df)

In [0]:
movies_df = movies_df.join(join_country_df, movies_df.movie_id == join_country_df.movie_id, "inner")\
                     .select(movies_df.year_release_date, movies_df.budget, movies_df.revenue, join_country_df.country_name)
display(movies_df)

In [0]:
from pyspark.sql.functions import dense_rank, desc, sum, lit
from pyspark.sql.window import Window

movies_df = movies_df.groupBy("year_release_date", "country_name")\
                     .agg(
                        sum("budget").alias("total_budget"), 
                        sum("revenue").alias("total_revenue")
                        )

window_rank_desc = Window.partitionBy("year_release_date").orderBy(desc("total_budget"), desc("total_revenue"))
final_df = movies_df.withColumn("rank", dense_rank().over(window_rank_desc))\
   .withColumn("created_date", lit(v_file_date))
                    

In [0]:
merge_condition = 'tgt.year_release_date = src.year_release_date AND tgt.country_name = src.country_name AND tgt.created_date = src.created_date'
merge_delta_lake(final_df, "movie_gold", "results_group_movie_country", gold_folder_path, merge_condition, "created_date")

In [0]:
%sql
SELECT * FROM movie_gold.results_group_movie_country